In [2]:
import sys

In [ ]:
#!{sys.executable} -m pip install --upgrade pip

In [ ]:
#!{sys.executable} -m pip install --user --upgrade kfp==2.0.0b13

In [8]:
!cat ./requirements.txt

huggingface_hub==0.16.4
transformers==4.32.1
torch==2.0.1
accelerate==0.22.0
#huggingface_hub use_auth_token need this.
urllib3==2.0.4 
jsonschema==4.19.0
fastai==2.7.12
torchaudio==2.0.2
torchvision==0.15.2
#argparse==1.4.0
click==8.1.7
#kfp==1.8.22

In [9]:
!{sys.executable} -m pip install --user --upgrade -r ./requirements.txt

  Obtaining dependency information for click==8.1.7 from https://files.pythonhosted.org/packages/00/2e/d53fa4befbf2cfa713304affc7ca780ce4fc1fd8710527771b58311a3229/click-8.1.7-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 4.8 MB/s eta 0:00:00


## (optional) restart kernel

### (optional) Set huggingface cli in terminal

```shell
PATH=${PATH}:/home/jupyter/.local/bin
```

In [5]:
# (optional) uncomment the following lines to set path in python notebook cell for notebook session 
# PATH=%env PATH
# %env PATH={PATH}:/home/jupyter/.local/bin